In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from google.colab import files

In [2]:
# Upload dataset
uploaded = files.upload()
import pandas as pd

# Load dataset
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name, header=None)

Saving spambase.data to spambase.data


In [3]:
# Splitting features and target
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values  # Target

In [4]:
# Standardizing the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Creating DataLoader for batch processing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=12)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=12)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
# Define RNN model class
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, pooling_type):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.pooling_type = pooling_type
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        if self.pooling_type == 'max':
            out = torch.max(out, dim=1).values
        elif self.pooling_type == 'avg':
            out = torch.mean(out, dim=1)
        out = self.fc(out)
        return out

In [7]:
# Define training function
def train_model(model, train_loader, optimizer, criterion, num_epochs, scheduler=None):
    model.train()
    for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch.unsqueeze(1))  # Adding sequence dimension
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        if scheduler:
            scheduler.step()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_loader):.4f}")

In [8]:
# Define evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch.unsqueeze(1))  # Adding sequence dimension
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    return accuracy

In [9]:
# Parameters
input_size = X_train.shape[1]
output_size = len(np.unique(y))
hidden_sizes = [32, 64]
pooling_types = ['max', 'avg']
optimizers = {'SGD': optim.SGD, 'RMSProp': optim.RMSprop, 'Adam': optim.Adam}
epochs_list = [5, 50, 100, 250, 350]

In [10]:
# Results dictionary
results = {
    'hidden_size': {},
    'pooling_type': {},
    'optimizer': {},
    'epochs': {}
}

In [11]:
# Experiment with hidden sizes
for hidden_size in hidden_sizes:
    print(f"\nExperimenting with Hidden Size: {hidden_size}")
    model = RNNModel(input_size, hidden_size, num_layers=1, output_size=output_size, pooling_type='avg')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    train_model(model, train_loader, optimizer, criterion, num_epochs=50, scheduler=scheduler)
    accuracy = evaluate_model(model, test_loader)
    results['hidden_size'][hidden_size] = accuracy


Experimenting with Hidden Size: 32


Training Epochs:   0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Training Epochs:   2%|▏         | 1/50 [00:01<00:53,  1.09s/it]

Epoch 1/50, Loss: 0.2526


Training Epochs:   4%|▍         | 2/50 [00:01<00:46,  1.03it/s]

Epoch 2/50, Loss: 0.1861


...

Training Epochs:  98%|█████████▊| 49/50 [00:34<00:00,  1.31it/s]

Epoch 49/50, Loss: 0.0715


Training Epochs: 100%|██████████| 50/50 [00:35<00:00,  1.41it/s]

Epoch 50/50, Loss: 0.0713


Test Accuracy: 92.62%


In [12]:
# Experiment with pooling types
for pooling_type in pooling_types:
    print(f"\nExperimenting with Pooling Type: {pooling_type}")
    model = RNNModel(input_size, 32, num_layers=1, output_size=output_size, pooling_type=pooling_type)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    train_model(model, train_loader, optimizer, criterion, num_epochs=50, scheduler=scheduler)
    accuracy = evaluate_model(model, test_loader)
    results['pooling_type'][pooling_type] = accuracy


Experimenting with Pooling Type: max


Training Epochs:   2%|▏         | 1/50 [00:00<00:30,  1.59it/s]

Epoch 1/50, Loss: 0.2588


Training Epochs:   4%|▍         | 2/50 [00:01<00:30,  1.57it/s]

Epoch 2/50, Loss: 0.1903


...

Training Epochs:  98%|█████████▊| 49/50 [00:36<00:00,  1.26it/s]

Epoch 49/50, Loss: 0.0766


Training Epochs: 100%|██████████| 50/50 [00:38<00:00,  1.31it/s]

Epoch 50/50, Loss: 0.0776


Test Accuracy: 93.49%


In [13]:
# Experiment with optimizers
for optimizer_name, optimizer_class in optimizers.items():
    print(f"\nExperimenting with Optimizer: {optimizer_name}")
    model = RNNModel(input_size, 32, num_layers=1, output_size=output_size, pooling_type='avg')
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer_class(model.parameters(), lr=0.01)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    train_model(model, train_loader, optimizer, criterion, num_epochs=50, scheduler=scheduler)
    accuracy = evaluate_model(model, test_loader)
    results['optimizer'][optimizer_name] = accuracy


Experimenting with Optimizer: SGD


Training Epochs:   2%|▏         | 1/50 [00:01<00:53,  1.10s/it]

Epoch 1/50, Loss: 0.6122


Training Epochs:   4%|▍         | 2/50 [00:01<00:42,  1.14it/s]

Epoch 2/50, Loss: 0.4982


...

Training Epochs:  98%|█████████▊| 49/50 [00:48<00:00,  1.11it/s]

Epoch 49/50, Loss: 0.0745


Training Epochs: 100%|██████████| 50/50 [00:50<00:00,  1.00s/it]

Epoch 50/50, Loss: 0.0748


Test Accuracy: 92.83%


In [14]:
# Experiment with epochs
for num_epochs in epochs_list:
    print(f"\nExperimenting with Epochs: {num_epochs}")
    model = RNNModel(input_size, 32, num_layers=1, output_size=output_size, pooling_type='avg')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    train_model(model, train_loader, optimizer, criterion, num_epochs=num_epochs, scheduler=scheduler)
    accuracy = evaluate_model(model, test_loader)
    results['epochs'][num_epochs] = accuracy


Experimenting with Epochs: 5


Training Epochs:  20%|██        | 1/5 [00:00<00:03,  1.14it/s]

Epoch 1/5, Loss: 0.2570


Training Epochs:  40%|████      | 2/5 [00:01<00:02,  1.18it/s]

Epoch 2/5, Loss: 0.1899


...

Training Epochs: 100%|█████████▉| 349/350 [08:19<00:01,  1.47s/it]

Epoch 349/350, Loss: 0.0728


Training Epochs: 100%|██████████| 350/350 [08:20<00:00,  1.43s/it]

Epoch 350/350, Loss: 0.0721


Test Accuracy: 93.27%


In [15]:
# Display results
print("\nExperiment Results:")
for key, value in results.items():
    print(f"{key}: {value}")


Experiment Results:
hidden_size: {32: 0.9381107491856677, 64: 0.9261672095548317}
pooling_type: {'max': 0.9391965255157437, 'avg': 0.9348534201954397}
optimizer: {'SGD': 0.9109663409337676, 'RMSProp': 0.9283387622149837, 'Adam': 0.9283387622149837}
epochs: {5: 0.9348534201954397, 50: 0.9326818675352877, 100: 0.9326818675352877, 250: 0.9413680781758957, 350: 0.9326818675352877}
